In [ ]:
# ↓Colabを立ち上げるたび、パッケージをインストールする
!pip install janome

In [ ]:
# ↓Colabを立ち上げるたび、パッケージをインストールする
!apt-get -y install fonts-ipafont-gothic

In [ ]:
# Twitterのフォロワーを取得するためのスクリプト
# coding:utf-8
import tweepy
import pandas as pd
import time
#import os

# Oauth
#CONSUMER_KEY = os.environ['CONSUMER_KEY']
#CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
#ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
#ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

# ↓Twitterの認証情報を入力する
CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""

# Oauth
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit = True)

# dataframe
columns = ["screen_name", "username", "description", "follower", "following"]
df = pd.DataFrame(columns=columns)

# ↓取得したいTwitterユーザーのscreen_nameを入力する
# target_user
target_screen_name = "#####"

# followers
followers_ids = tweepy.Cursor(api.followers_ids, id = target_screen_name, cursor = -1).items()
followers_ids_list = []
try:
  for followers_id in followers_ids:
    followers_ids_list.append(followers_id)
except tweepy.error.TweepError as e:
  print (e.reason)

# save dataframe
for follower_id in followers_ids_list:
 try:
    user = api.get_user(follower_id)
    screen_name = user.screen_name
    username = user.name
    description = user.description
    follower = user.followers_count
    following = user.friends_count
    se = pd.Series([screen_name, username, description, follower, following], columns)
    df = df.append(se, ignore_index=True)
    print(df)
 except Exception as e:
    print(e)

# ↓CSVデータを出力、ファイル名を入力する（拡張子は.csv）
# export CSV
filename = "#####.csv"
df.to_csv(filename, encoding = 'utf-8-sig')

In [ ]:
# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# フォロワー情報を元にWordCloudを作成するスクリプト
# coding:utf-8
import csv
from janome.tokenizer import Tokenizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from bs4 import BeautifulSoup
from collections import Counter, defaultdict

# count 名詞
def counter(texts):
    t = Tokenizer()
    words_count = defaultdict(int)
    words = []
    for text in texts:
        tokens = t.tokenize(text)
        for token in tokens:
            # export 名詞
            pos = token.part_of_speech.split(",")[0]
            if pos in ["名詞"]:
                #↓含めたくない単語を入力する
                # ng word
                if token.base_form not in [u'てる', u'いる', u'なる', u'れる', u'する', u'ある', u'こと', u'これ', u'さん', u'して', \
             u'くれる', u'やる', u'くださる', u'そう', u'せる', u'した',  u'思う',  \
             u'それ', u'ここ', u'ちゃん', u'くん', u'', u'て',u'に',u'を',u'は',u'の', u'が', u'と', u'た', u'し', u'で', \
             u'ない', u'も', u'な', u'い', u'か', u'ので', u'よう', u'']:
                    words_count[token.base_form] += 1
                    words.append(token.base_form)
    return words_count, words

#↓ファイルを指定する
with open("drive/My Drive/#####.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    texts = []
    for row in reader:
        if(len(row) > 0):
            text = row[0].split("http")
            texts.append(text[0])

words_count, words = counter(texts)
text = " ".join(words)

# default font
fpath = "/usr/share/fonts/truetype/fonts-japanese-gothic.ttf"
wordcloud = WordCloud(
                      font_path=fpath,
                      width=900, height=500,
                      background_color="white",
                      max_words=500,
                      min_font_size=4,
                      collocations = False
                      ).generate(text)

# ↓WordCloudの結果を出力、ファイル名を入力する（拡張子は.png）
wordcloud.to_file("./#####.png")